In [1]:
import pandas as pd

from pathlib import Path

In [2]:
anno_path = Path("../human_annotation/new_human_annotations/")

In [3]:
all_anno = []
for file in anno_path.glob('*.csv'):
    anno_name_split = file.stem.split("_")
    anno_name = anno_name_split[0].lower()
    anno = pd.read_csv(file, index_col=[0])
    anno['annotator'] = anno_name
    anno['split'] = anno_name_split[-1]
    all_anno.append(anno)
    print(anno_name, anno.shape, anno['changed'].sum())

nico (300, 20) 300
silpa (300, 20) 300
zhaochong (300, 20) 300
srishti (300, 20) 300
srishti (300, 20) 300
srishti (300, 20) 300
peter (300, 20) 300
weronika (300, 20) 300
arnav (300, 20) 300
sander (300, 20) 300


In [4]:
all_anno = pd.concat(all_anno)

In [5]:
print(all_anno.shape)
print(all_anno.columns)

(3000, 20)
Index(['example', 'changed', 'quality of life', 'capacity and resources',
       'political', 'security and defense',
       'policy prescription and evaluation', 'fairness and equality', 'none',
       'health and safety', 'legality, constitutionality and jurisprudence',
       'morality', 'economic', 'public opinion', 'cultural identity',
       'external regulation and reputation', 'crime and punishment', 'uuid',
       'annotator', 'split'],
      dtype='object')


In [7]:
all_anno['uuid'].nunique()

1078

In [10]:
splits = all_anno['split'].unique()
for split1 in splits:
    for split2 in splits:
        if split1 != split2:
            split1_uuids = set(all_anno[all_anno['split'] == split1]['uuid'])
            split2_uuids = set(all_anno[all_anno['split'] == split2]['uuid'])
            overlap = split1_uuids.intersection(split2_uuids)
            print(f"Overlap between {split1} and {split2}: {len(overlap)}")

Overlap between 4 and 3: 0
Overlap between 4 and 2: 0
Overlap between 4 and 1: 43
Overlap between 3 and 4: 0
Overlap between 3 and 2: 0
Overlap between 3 and 1: 47
Overlap between 2 and 4: 0
Overlap between 2 and 3: 0
Overlap between 2 and 1: 32
Overlap between 1 and 4: 43
Overlap between 1 and 3: 47
Overlap between 1 and 2: 32


In [7]:
label_cols = all_anno.columns[2:-3]

In [8]:
for anno_name, anno_df in all_anno.groupby('annotator'):
    print(f"Annotator: {anno_name}, Split: {anno_df['split'].unique()[0]}")
    print(f"No of annotations: {anno_df['changed'].sum()}")
    print(f"Unique uuids: {anno_df['uuid'].nunique()}")
    print(f"Average annotations per label: {anno_df[label_cols].sum(axis=1).mean():.2f}")
    print("------------")

Annotator: arnav, Split: 2
No of annotations: 300
Unique uuids: 300
Average annotations per label: 1.48
------------
Annotator: nico, Split: 4
No of annotations: 300
Unique uuids: 300
Average annotations per label: 1.00
------------
Annotator: peter, Split: 3
No of annotations: 300
Unique uuids: 300
Average annotations per label: 2.30
------------
Annotator: sander, Split: 4
No of annotations: 300
Unique uuids: 300
Average annotations per label: 1.12
------------
Annotator: silpa, Split: 3
No of annotations: 300
Unique uuids: 300
Average annotations per label: 1.15
------------
Annotator: srishti, Split: 2
No of annotations: 900
Unique uuids: 821
Average annotations per label: 1.17
------------
Annotator: weronika, Split: 2
No of annotations: 300
Unique uuids: 300
Average annotations per label: 1.06
------------
Annotator: zhaochong, Split: 2
No of annotations: 300
Unique uuids: 300
Average annotations per label: 1.07
------------


In [11]:
for split in splits:
    split_df = all_anno[all_anno['split'] == split]
    print(f"Split: {split}")
    if split_df['annotator'].nunique() > 1:
        annotators = split_df['annotator'].unique()
        for i in range(len(annotators)):
            for j in range(i + 1, len(annotators)):
                anno_1 = split_df[split_df['annotator'] == annotators[i]]
                anno_2 = split_df[split_df['annotator'] == annotators[j]]
                anno_1_labels = [set(row[row == True].index.tolist()) for index, row in anno_1[label_cols].iterrows()]
                anno_2_labels = [set(row[row == True].index.tolist()) for index, row in anno_2[label_cols].iterrows()]
                label_overlap = [len(a1.intersection(a2)) for a1, a2 in zip(anno_1_labels, anno_2_labels)]
                binary_overlap = [1 if overlap > 0 else 0 for overlap in label_overlap]
                print(f"Average label overlap between annotators {annotators[i]} and {annotators[j]}: {sum(label_overlap) / len(label_overlap):.2f}")
                print(f"Average binary overlap between annotators {annotators[i]} and {annotators[j]}: {sum(binary_overlap) / len(binary_overlap):.2f}")
    else:
        print("Only one annotator")
    print("-----------------")

Split: 4
Average label overlap between annotators nico and sander: 0.45
Average binary overlap between annotators nico and sander: 0.45
-----------------
Split: 3
Average label overlap between annotators silpa and srishti: 0.49
Average binary overlap between annotators silpa and srishti: 0.48
Average label overlap between annotators silpa and peter: 0.44
Average binary overlap between annotators silpa and peter: 0.43
Average label overlap between annotators srishti and peter: 0.46
Average binary overlap between annotators srishti and peter: 0.44
-----------------
Split: 2
Average label overlap between annotators zhaochong and srishti: 0.65
Average binary overlap between annotators zhaochong and srishti: 0.65
Average label overlap between annotators zhaochong and weronika: 0.63
Average binary overlap between annotators zhaochong and weronika: 0.63
Average label overlap between annotators zhaochong and arnav: 0.69
Average binary overlap between annotators zhaochong and arnav: 0.69
Averag

In [12]:
# Combine labels for split 2 and save as a validation set
split_2_srishti = all_anno[(all_anno['split'] == '2') & (all_anno['annotator'] == 'srishti')]
split_2_weronika = all_anno[(all_anno['split'] == '2') & (all_anno['annotator'] == 'weronika')]
split_2_arnav = all_anno[(all_anno['split'] == '2') & (all_anno['annotator'] == 'arnav')]
print(len(split_2_srishti), len(split_2_weronika))


300 300


In [13]:
split_2_srishti['labels'] = [set(row[row == True].index.tolist()) for _, row in split_2_srishti[label_cols].iterrows()]
split_2_weronika['labels'] = [set(row[row == True].index.tolist()) for _, row in split_2_weronika[label_cols].iterrows()]
split_2_arnav['labels'] = [set(row[row == True].index.tolist()) for _, row in split_2_arnav[label_cols].iterrows()]

/tmp/ipykernel_253496/4220838887.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_2_srishti['labels'] = [set(row[row == True].index.tolist()) for _, row in split_2_srishti[label_cols].iterrows()]
/tmp/ipykernel_253496/4220838887.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_2_weronika['labels'] = [set(row[row == True].index.tolist()) for _, row in split_2_weronika[label_cols].iterrows()]
/tmp/ipykernel_253496/4220838887.py:3: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [14]:
merged_set2 = split_2_srishti[['uuid','labels']].merge(split_2_weronika[['uuid','labels']], on='uuid', suffixes=('_srishti', '_weronika'))
merged_set2 = merged_set2.merge(split_2_arnav[['uuid','labels']], on='uuid')

In [17]:
merged_set2['merged_labels'] = merged_set2.apply(lambda x: x['labels_srishti'].union(x['labels_weronika']), axis=1)
merged_set2['merged_labels'] = merged_set2.apply(lambda x: x['merged_labels'].union(x['labels']), axis=1)

In [21]:
# merged_set2[['uuid', 'merged_labels']].to_csv("../data/human/val_set.csv", index=False)
merged_set2[['uuid', 'merged_labels']].sample(30).to_csv("../data/human/val_set_sample.csv", index=False)

In [20]:
merged_set2[['uuid', 'merged_labels']].to_csv("../data/human/val_set_full.csv", index=False)

In [20]:
merged_set2['merged_labels'].apply(len).value_counts()

merged_labels
1    161
2    105
3     27
4      5
5      1
6      1
Name: count, dtype: int64

In [18]:
sample_df = pd.read_csv("../human_annotation/sample_data_anno.csv")
sample_df['uuid'].nunique()

2300